<p style="float: left;"><a href="variances.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="p2-exercises.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

## Lower type bounds

- Remember [upper type bounds](upper-type-bounds.ipynb) for type parameters. 

    - In a type parameter declaration such as `T <: U`.
      
    - The type parameter **`T` is restricted to range only over <span style="color:red">subtypes</span> of type `U`**.

- Symmetrical to _upper bounds_ are _lower bounds_ in Scala. 

    - In a type parameter declaration `T >: S`.
      
    - The type parameter **`T` is restricted to range only over <span style="color:red">supertypes</span> of type `S`**.

- Using _lower bounds_, we can generalize the `push` method in `Stack` as follows:

    ```scala
    class Stack[+A] {
        def push[B >: A](elem: B): Stack[B] = . . .
    }
    
    ```

    <br/>

    - This solves the variance problem since **the type parameter `A` appears no longer in contravariance position**.

    - Instead, `A` appears as _lower type bound_ for another type parameter, `B`. **The compiler must respect:**
     
        1) Requires `B` to be a supertype of the current element type `A`.
           
        2) The argument of type `B` must to accept the actual argument passed.

        In practice, compiler picks the **least upper bound (LUB)** of `A` and the argument’s type. <span style="color:red">**Updating the a stack’s type accordingly to `Stack[B]`</span>.**

        ```scala
        trait Stack[+A]:
          def push[B >: A](elem: B): Stack[B] = . . .
        
        val s: Stack[Int] = . . .
        val s2 = s.push("hello")   // inferred B = AnyVal, s2: Stack[AnyVal]

        ```




## Union types

Scala 3 introduced the union type (`A | B`).

- <span style="color:red">**When you push an element of type `B` onto a `Stack[A]`, the resulting stack has the type `Stack[A | B]` instead of `Stack[B]`</span>.**

In [2]:
abstract class Stack[+A]:
    def push[B](x: B): Stack[A | B] = new NonEmptyStack[A | B](x, this)
    def isEmpty: Boolean
    def top: A
    def pop: Stack[A]

class EmptyStack[A] extends Stack[A]:
    def isEmpty = true
    def top = throw new Exception("EmptyStack.top")
    def pop = throw new Exception("EmptyStack.pop")
    override def toString: String = "--()"

class NonEmptyStack[A](elem: A, rest: Stack[A]) extends Stack[A]:
    def isEmpty = false
    def top = elem
    def pop = rest
    override def toString(): String =
        "--" ++ top.toString() ++ "--<" ++ pop.toString()

defined class Stack
defined class EmptyStack
defined class NonEmptyStack

In [3]:
val stackInt = new EmptyStack().push(1 : Int)

stackInt: Stack[Int] = --1--<--()

In [4]:
val stackDouble = stackInt.push(1 : Double)

stackDouble: Stack[ammonite.$sess.cmd3.wrapper.cmd2.Stack[scala.Int | scala.Double]] = --1.0--<--1--<--()

In [17]:
new EmptyStack().push(1: Int).push(1 : Double).push('c')

res17: Stack[cmd17.this.cmd8.Stack[scala.Int | scala.Double | scala.Char]] = --c--<--1.0--<--1--<--()

## Lower type bounds vs union types.

More **precision** in types. With _lower type bounds_ and **LUB (least upper bound, Scala 2 style), the compiler often jumps too far up the type hierarchy.**

```scala
val xs = List(1, "hi")
// Scala 2: List[Any]
// Scala 3: List[Int | String]
```

<br/>

- `List[Any]`: you lose track of what’s inside, everything is “just Any.”

- `List[Int | String]`: the compiler still knows it can *only* be `Int` or `String`.

That extra precision means:

- You can safely pattern match later without casting.

- The compiler warns you if you forget a case.


### Safer code through exhaustiveness

- Union types allow the compiler to check *all possibilities*:

    ```scala
    def size(x: Int | String): Int =
      x match
        case i: Int    => i
        case s: String => s.length
    ```

### No need for “artificial” bounds

- Old style:

    ```scala
    def push[B >: A](elem: B): Stack[B]
    ```

    <br/>
    
- New style:

    ```scala
    def push[B](elem: B): Stack[A | B]
    ```

    <br/>

    - **You don’t have to reason about variance bounds (`B >: A`).** Instead, the semantics are clearer: *“new stack contains elements of either the old type or the new one.”*


### You can still widen when you want

- If you don’t want to carry around `Int | String | Double | Foo | Bar`, you can always widen:

    ```scala
    val xs: List[AnyVal] = List(1).appended(3.14)
    ```

    <br/>

    **So you don’t lose the old behaviour - you gain the option of being precise by default.**


<p style="float: left;"><a href="variances.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="p2-exercises.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>